In [111]:
import re
import collections
import os
import time

In [112]:
def tokenize(f):
    #Returns [(term, pos), (term, pos) ...]
    terms = []
    with open("collection/" + f) as file:
        line = file.readline()
        while line:
            # Regex to match only strings and spaces 
            line = re.sub(r'[^A-Za-z\s]+', '', line)
            for i, word in enumerate(line.split()):
                terms.append((word.lower(), file.tell()+i))
            line = file.readline() 
    return terms

In [113]:
def createPostingList():
    postingList = collections.defaultdict(list)
    curDocID = 0

    for f in os.listdir("collection"):
        terms = tokenize(f)

        # create map {word:[pos1, pos2, pos3]}
        wordToPos = collections.defaultdict(list)
        for word, pos in terms:
            wordToPos[word].append(pos)

        # append to posting list {term : [(docID1, [pos1, pos2, pos3, pos4])]}
        for term, arr in wordToPos.items():
            postingList[term].append((curDocID, wordToPos[term]))

        # For every file update id 
        curDocID += 1
    
    return postingList
    

In [114]:
startTime = time.time()
result = createPostingList()
endTime = time.time()

In [115]:
print(f"Index built in {endTime - startTime} seconds.")

Index built in 0.7736330032348633 seconds.


In [158]:
def mergeTwo(A, B):
    if not B:
        return [i for i, arr in A]

    a = b = 0
    res = []
    while a < len(A) and b < len(B):
        if A[a][0] > B[b][0]:
            b += 1
        elif A[a][0] < B[b][0]:
            a += 1
        else:
            res.append((A[a][0], []))
            a += 1
            b += 1
    return res

In [166]:
query = "the cat"
fixedQuery = re.sub(r'[^A-Za-z\s]+', '', query)
queryTerms = fixedQuery.lower().split()

postingLists = [result[q] for q in queryTerms]

while len(postingLists) > 1:
    temp = []
    for i in range(0, len(postingLists), 2):
        temp.append(mergeTwo(postingLists[i], postingLists[i+1]))
    postingLists = temp
print(postingLists)


[[(37, []), (41, [])]]
